You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [25]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [26]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Latest Trends in Artificial Intelligence: A Comprehensive Guide

Introduction:
- Definition of Artificial Intelligence
- Importance of staying updated on AI trends
- Overview of key players in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- Autonomous vehicles and robotics
- AI in healthcare and finance

2. Key Players in the AI Industry
- Google DeepMind
- IBM Watson
- Mic

I now can give a great answer

Final Answer:

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction to Artificial Intelligence
Artificial Intelligence, commonly known as AI, is an ever-evolving field that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence. From advancements in machine learning to developments in natural language processing, AI technology is constantly pushing boundaries and revolutionizing various industries. It is essential for tech enthusiasts, industry professionals, students, and business owners to stay informed about the latest trends in AI to comprehend the impact of this cutting-edge technology on society and the workforce.

## Latest Trends in AI
In recent years, AI has witnessed significant progress in machine learning, enabling machines to learn from data and enhance their performance without explicit programming. Natural language processing has also made remarkable a

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction to Artificial Intelligence
Artificial Intelligence, commonly known as AI, is an ever-evolving field that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence. From advancements in machine learning to developments in natural language processing, AI technology is constantly pushing boundaries and revolutionizing various industries. It is essential for tech enthusiasts, industry professionals, students, and business owners to stay informed about the latest trends in AI to comprehend the impact of this cutting-edge technology on society and the workforce.

## Latest Trends in AI
In recent years, AI has witnessed significant progress in machine learning, enabling machines to learn from data and enhance their performance without explicit programming. Natural language processing has also made remarkable advancements, allowing machines to comprehend, interpret, and generate human language. The rise of autonomous vehicles and robotics is transforming transportation and manufacturing industries. Additionally, AI applications in healthcare and finance are streamlining decision-making processes and elevating patient care and financial services.

## Key Players in the AI Industry
Numerous key players in the AI industry are leading the charge in innovation and research. Google DeepMind, renowned for its AlphaGo program, has made substantial contributions to AI research. IBM Watson is a trailblazer in cognitive computing and offers AI solutions for businesses. Microsoft Azure provides a wide array of AI tools and services to empower organizations with AI capabilities. Tesla, with its self-driving technology, is spearheading autonomous vehicles and AI integration.

## Noteworthy News in AI
Recent breakthroughs in AI research, such as OpenAI's GPT-3 language model, have captivated attention due to their impressive capabilities. The influence of AI on various industries, ranging from retail to manufacturing, is reshaping business operations and enhancing customer experiences. Ethical considerations in AI development, including bias in algorithms and data privacy concerns, are becoming increasingly crucial topics for discussion and regulation within the AI community.

## Conclusion
As AI technology continues to progress, it is imperative for individuals and businesses to stay abreast of the latest trends and advancements in the field. By familiarizing themselves with the key players in the AI industry, keeping up with noteworthy news in AI, and exploring case studies of AI applications, readers can gain valuable insights into the future prospects of AI technology. Let's embrace the opportunities presented by AI while also reflecting on the ethical implications and societal impacts of this transformative technology.

In conclusion, the realm of Artificial Intelligence is in a constant state of evolution, and staying informed about the latest trends and developments is essential for all those interested in this field. By delving into the key players in the AI industry, understanding noteworthy news in AI, and examining case studies and examples of AI applications, we can all contribute to shaping the future of AI technology. Embrace the possibilities of AI while also contemplating the ethical considerations and societal impacts to ensure a responsible and sustainable integration of AI into our lives.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Rise of Sustainable Fashion: Trends, Players, and News

Introduction:
- Brief overview of the growing popularity of sustainable fashion
- Importance of making informed decisions as consumers

Key Points:
1. Latest Trends in Sustainable Fashion
- Rise of eco-friendly materials such as organic cotton, hemp, and recycled polyester
- Adoption of circular fashion practices to reduce waste
- Emphasis on ethical labor practices and transparency in the supply chain

2. Key Players in the Sustainable Fashion Indus

I now can give a great answer

Final Answer:
# The Rise of Sustainable Fashion: Trends, Players, and News

In recent years, there has been a significant shift towards sustainable fashion as consumers become more conscious of the environmental and social impact of their clothing choices. With the rise of fast fashion and its detrimental effects on the planet, the importance of making informed decisions as consumers has never been more critical. By understanding the latest trends, key players, and noteworthy news in the sustainable fashion industry, individuals can support ethical and eco-friendly practices in the fashion world.

## Latest Trends in Sustainable Fashion

One of the most significant trends in sustainable fashion is the use of eco-friendly materials such as organic cotton, hemp, and recycled polyester. These materials not only reduce the environmental impact of clothing production but also promote a more sustainable future for the industry. Additionally, the adoption of cir

In [14]:
Markdown(result)

# The Rise of Sustainable Fashion: Trends, Players, and News

In recent years, there has been a significant shift towards sustainable fashion as consumers become more conscious of the environmental and social impact of their clothing choices. With the rise of fast fashion and its detrimental effects on the planet, the importance of making informed decisions as consumers has never been more critical. By understanding the latest trends, key players, and noteworthy news in the sustainable fashion industry, individuals can support ethical and eco-friendly practices in the fashion world.

## Latest Trends in Sustainable Fashion

One of the most significant trends in sustainable fashion is the use of eco-friendly materials such as organic cotton, hemp, and recycled polyester. These materials not only reduce the environmental impact of clothing production but also promote a more sustainable future for the industry. Additionally, the adoption of circular fashion practices, which focus on reducing waste and extending the lifespan of clothing, has gained momentum. This shift towards a more circular economy is essential in minimizing the environmental footprint of the fashion industry. Furthermore, there is an emphasis on ethical labor practices and transparency in the supply chain, ensuring that workers are treated fairly and that consumers are aware of the production processes behind their clothing.

## Key Players in the Sustainable Fashion Industry

Leading sustainable fashion brands such as Patagonia, Eileen Fisher, and Stella McCartney have paved the way for ethical and eco-conscious practices in the industry. These brands prioritize sustainability in every aspect of their business, from sourcing materials to production processes. Moreover, there has been a rise in collaboration between mainstream brands and sustainable fashion labels, showcasing a growing interest in sustainability from all sectors of the industry. Influencers and celebrities also play a significant role in promoting sustainable fashion, using their platforms to raise awareness and encourage their followers to make more conscious clothing choices.

## Noteworthy News in Sustainable Fashion

Legislation and regulations promoting sustainability in the fashion industry have been on the rise, pushing brands to adopt more sustainable practices. Additionally, the impact of COVID-19 has highlighted the importance of sustainability in the fashion industry, with consumers reevaluating their shopping habits and supporting brands that prioritize ethical and eco-friendly practices. Success stories of sustainable fashion brands and initiatives serve as inspiration for the industry, showing that it is possible to create fashionable and sustainable clothing without compromising on style or quality.

In conclusion, the rise of sustainable fashion is a positive shift towards a more ethical and eco-conscious industry. By staying informed about the latest trends, key players, and noteworthy news in sustainable fashion, consumers can make a positive impact through their clothing choices. It is essential to support brands that prioritize sustainability and to advocate for a more transparent and ethical fashion industry. Let's embrace sustainable fashion and contribute to a more sustainable future for all.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).